In [3]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [32]:
basedir = './국토교통부_건축물대장_층별개요/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['DJY_MART_03_201412.txt',
 'mart_djy_04_201612.txt',
 'mart_djy_04_201712.txt',
 'mart_djy_04_201812.txt',
 'mart_djy_04_201912.txt',
 'mart_djy_04_202011.txt',
 'MART_DJY_FLR_OULN_201512.txt']

In [5]:
for file in filenames:
    print(file)
    f = open(basedir + file, 'r')
    print(f.readline())
    f.close()

DJY_MART_03_201412.txt
마트_층별_개요_PK,건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블럭,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명칭,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적,주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자

mart_djy_04_201612.txt
26530-100175951|부산광역시 사상구 주례동 67-9번지|부산광역시 사상구 주례로9번길 11||26530|10600|0|0067|0009||||265304217356|10601|0|11|0|주건축물제1동|20|지상|1|1층|21|철근콘크리트구조|철근콘크리트구조|04402|사무소|제2종근린생활시설(사무소(주차장)|23.04|0|주건축물|0|20100105

mart_djy_04_201712.txt
ORACLEDUMP	|MGM_BLDRGST_PK	PLAT_PLC	NEW_PLAT_PLC	BLD_NM	SIGUNGU_CD	BJDONG_CD	PLAT_GB_CD	BUN	JI	SPLOT_NM	BLOCK	LOT	NA_ROAD_CD	NA_BJDONG_CD	NA_UGRND_CD	NA_MAIN_BUN	NA_SUB_BUN	DONG_NM	FLR_GB_CD	FLR_GB_CD_NM	FLR_NO	FLR_NO_NM	STRCT_CD	STRCT_CD_NM	ETC_STRCT	MAIN_PURPS_CD	MAIN_PURPS_CD_NM	ETC_PURPS	AREA	MAIN_ATCH_GB_CD	MAIN_ATCH_GB_CD_NM	AREA_EXCT_YN	CRTN_DAY

mart_djy_04_201812.txt
29155-8546|광주광역시 남구 양촌동 358번지| 광주광역시 남구 포충로 125-5||2

In [6]:
f = open(basedir + filenames[0], 'r')

In [7]:
f.readline()

'마트_층별_개요_PK,건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블럭,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명칭,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적,주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자\n'

In [8]:
f.readline()

'"41465-1290","41465-10023034802","경기도 용인시 수지구 성복동 759번지",,"버들치마을 성복 힐스테이트 2차","41465","10600","0","0759","0000",,,,,,,,,"414654415063","10601","0",13,0,"223","20","지상",1,"1층","21","철근콘크리트구조","철근콘크리트구조","02001","아파트","경로당",216.785,"1","부속건축물","0","20141109"\n'

In [9]:
f.close()

In [11]:
chunks = pd.read_csv(basedir + filenames[0], chunksize=1, encoding='euc-kr')

In [12]:
prac_df = next(chunks)
prac_df

,마트_층별_개요_PK,건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블럭,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명칭,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적,주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,41465-1290,41465-10023034802,경기도 용인시 수지구 성복동 759번지,NaN,버들치마을 성복 힐스테이트 2차,41465,10600,0,759,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414654415063,10601,0,13,0,223,20,지상,1,1층,21,철근콘크리트구조,철근콘크리트구조,2001,아파트,경로당,216.785,1,부속건축물,0,20141109


In [19]:
prac_df.shape

(1, 39)

In [59]:
no_header_files_list = ['MART_DJY_FLR_OULN_201512.txt', 'mart_djy_04_201612.txt', 'mart_djy_04_201812.txt',
                        'mart_djy_04_201912.txt', 'mart_djy_04_202011.txt']

In [14]:
f = open(basedir + 'mart_djy_04_201712.txt', 'r')

In [15]:
f.readline()

'ORACLEDUMP\t|MGM_BLDRGST_PK\tPLAT_PLC\tNEW_PLAT_PLC\tBLD_NM\tSIGUNGU_CD\tBJDONG_CD\tPLAT_GB_CD\tBUN\tJI\tSPLOT_NM\tBLOCK\tLOT\tNA_ROAD_CD\tNA_BJDONG_CD\tNA_UGRND_CD\tNA_MAIN_BUN\tNA_SUB_BUN\tDONG_NM\tFLR_GB_CD\tFLR_GB_CD_NM\tFLR_NO\tFLR_NO_NM\tSTRCT_CD\tSTRCT_CD_NM\tETC_STRCT\tMAIN_PURPS_CD\tMAIN_PURPS_CD_NM\tETC_PURPS\tAREA\tMAIN_ATCH_GB_CD\tMAIN_ATCH_GB_CD_NM\tAREA_EXCT_YN\tCRTN_DAY\n'

In [16]:
f.readline()

'29155-14451|광주광역시 남구 주월동 974-81번지|광주광역시 남구 군분로23번길 8-1||29155|10600|0|0974|0081||||291554283074|10601|0|8|1||20|지상|1|1층|11|벽돌구조|시멘트벽돌|01003|다가구주택|1가구 주택|101.89|0|주건축물||20100105\n'

In [18]:
len(f.readline().split('|'))

33

In [20]:
f.close()

In [28]:
chunks = pd.read_csv(basedir + 'mart_djy_04_201712.txt', encoding='euc-kr', chunksize=1, sep='|', header=0, names=colnames)

In [29]:
prac_df = next(chunks)

In [30]:
print(prac_df.shape)
prac_df.head()

(1, 33)


,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,29155-14451,광주광역시 남구 주월동 974-81번지,광주광역시 남구 군분로23번길 8-1,NaN,29155,10600,0,974,81,NaN,NaN,NaN,291554283074,10601,0,8,1,NaN,20,지상,1,1층,11,벽돌구조,시멘트벽돌,1003,다가구주택,1가구 주택,101.89,0,주건축물,NaN,20100105


In [33]:
filenames

['DJY_MART_03_201412.txt',
 'mart_djy_04_201612.txt',
 'mart_djy_04_201712.txt',
 'mart_djy_04_201812.txt',
 'mart_djy_04_201912.txt',
 'mart_djy_04_202011.txt',
 'MART_DJY_FLR_OULN_201512.txt']

In [34]:
chunks = pd.read_csv(basedir + filenames[0], encoding='euc-kr', chunksize=1)

In [35]:
prac_2014_df = next(chunks)
prac_2014_df

,마트_층별_개요_PK,건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블럭,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명칭,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적,주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,41465-1290,41465-10023034802,경기도 용인시 수지구 성복동 759번지,NaN,버들치마을 성복 힐스테이트 2차,41465,10600,0,759,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414654415063,10601,0,13,0,223,20,지상,1,1층,21,철근콘크리트구조,철근콘크리트구조,2001,아파트,경로당,216.785,1,부속건축물,0,20141109


In [40]:
colnames

['관리_건축물대장_PK',
 '대지_위치',
 '도로명_대지_위치',
 '건물_명',
 '시군구_코드',
 '법정동_코드',
 '대지_구분_코드',
 '번',
 '지',
 '특수지_명',
 '블록',
 '로트',
 '새주소_도로_코드',
 '새주소_법정동_코드',
 '새주소_지상지하_코드',
 '새주소_본_번',
 '새주소_부_번',
 '동_명',
 '층_구분_코드',
 '층_구분_코드_명',
 '층_번호',
 '층_번호_명',
 '구조_코드',
 '구조_코드_명',
 '기타_구조',
 '주_용도_코드',
 '주_용도_코드_명',
 '기타_용도',
 '면적(㎡)',
 '주_부속_구분_코드',
 '주_부속_구분_코드_명',
 '면적_제외_여부',
 '생성_일자']

In [41]:
prac_2014_df.rename(columns={'건축물대장_PK':'관리_건축물대장_PK', '블럭':'블록', '동_명칭':'동_명', '면적':'면적(㎡)'}, inplace=True)

In [42]:
prac_2014_df

,마트_층별_개요_PK,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,41465-1290,41465-10023034802,경기도 용인시 수지구 성복동 759번지,NaN,버들치마을 성복 힐스테이트 2차,41465,10600,0,759,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414654415063,10601,0,13,0,223,20,지상,1,1층,21,철근콘크리트구조,철근콘크리트구조,2001,아파트,경로당,216.785,1,부속건축물,0,20141109


In [43]:
%%time
floor_info_2014_df = pd.read_csv(basedir + filenames[0], encoding='euc-kr')
floor_info_2014_df.rename(columns={'건축물대장_PK':'관리_건축물대장_PK', '블럭':'블록', '동_명칭':'동_명', '면적':'면적(㎡)'}, inplace=True)

<string>:2: DtypeWarning: Columns (10,11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 10.1 s


In [44]:
print(floor_info_2014_df.shape)
floor_info_2014_df.head()

(3663172, 39)


,마트_층별_개요_PK,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,41465-1290,41465-10023034802,경기도 용인시 수지구 성복동 759번지,NaN,버들치마을 성복 힐스테이트 2차,41465,10600,0,759,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.146544e+11,10601.0,0,13.0,0.0,223,20.0,지상,1,1층,21.0,철근콘크리트구조,철근콘크리트구조,02001,아파트,경로당,216.785,1.0,부속건축물,0.0,20141109
1,41465-1291,41465-10023034902,경기도 용인시 수지구 성복동 759번지,NaN,버들치마을 성복 힐스테이트 2차,41465,10600,0,759,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.146544e+11,10601.0,0,13.0,0.0,224,20.0,지상,1,1층,21.0,철근콘크리트구조,철근콘크리트구조,02001,아파트,문주,47.947,1.0,부속건축물,1.0,20141109
2,41610-1292,41610-10018964202,경기도 광주시 장지동 599-11번지,NaN,NaN,41610,11100,0,599,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-건축-신축허가-5,4.161044e+11,11101.0,0,59.0,0.0,NaN,20.0,지상,4,4층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,129.120,0.0,주건축물,0.0,20141109
3,41610-1293,41610-10018964202,경기도 광주시 장지동 599-11번지,NaN,NaN,41610,11100,0,599,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-건축-신축허가-5,4.161044e+11,11101.0,0,59.0,0.0,NaN,20.0,지상,3,3층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,129.120,0.0,주건축물,0.0,20141109
4,41610-1294,41610-10018964202,경기도 광주시 장지동 599-11번지,NaN,NaN,41610,11100,0,599,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-건축-신축허가-5,4.161044e+11,11101.0,0,59.0,0.0,NaN,30.0,옥탑,1,옥탑1층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,계단실,10.080,0.0,주건축물,1.0,20141109


In [45]:
seoul_2014_df = floor_info_2014_df[floor_info_2014_df['대지_위치'].str.contains('서울')]
seoul_2014_df.shape

ValueError: cannot index with vector containing NA / NaN values

In [46]:
nan_df = floor_info_2014_df[floor_info_2014_df['대지_위치'].isna()]

In [47]:
nan_df.shape

(44860, 39)

In [48]:
nan_df.head()

,마트_층별_개요_PK,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
1831007,41480-1830069,41480-2196302,NaN,NaN,진달래,41480,12300,0,199,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.148044e+11,10302.0,0,50.0,0.0,91동,10.0,지하,1,지1,21.0,철근콘크리트구조,철근콘크리트조,02003,다세대주택,다세대(3세대),116.97,0.0,주건축물,NaN,20141109
1831008,41480-1830070,41480-2196302,NaN,NaN,진달래,41480,12300,0,199,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.148044e+11,10302.0,0,50.0,0.0,91동,20.0,지상,1,1층,21.0,철근콘크리트구조,철근콘크리트조,02003,다세대주택,다세대(3세대),116.97,0.0,주건축물,NaN,20141109
1831009,41480-1830071,41480-2196302,NaN,NaN,진달래,41480,12300,0,199,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.148044e+11,10302.0,0,50.0,0.0,91동,20.0,지상,2,2층,21.0,철근콘크리트구조,철근콘크리트조,02003,다세대주택,다세대(3세대),117.46,0.0,주건축물,NaN,20141109
1831010,41480-1830072,41480-2196302,NaN,NaN,진달래,41480,12300,0,199,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.148044e+11,10302.0,0,50.0,0.0,91동,20.0,지상,3,3층,21.0,철근콘크리트구조,철근콘크리트조,02003,다세대주택,다세대(3세대),117.46,0.0,주건축물,NaN,20141109
1831011,41480-1830073,41480-2196302,NaN,NaN,진달래,41480,12300,0,199,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.148044e+11,10302.0,0,50.0,0.0,91동,20.0,지상,4,4층,21.0,철근콘크리트구조,철근콘크리트조,02003,다세대주택,다세대(3세대),113.86,0.0,주건축물,NaN,20141109


In [49]:
floor_info_2014_df.drop(nan_df.index, inplace=True)

In [50]:
seoul_2014_df = floor_info_2014_df[floor_info_2014_df['대지_위치'].str.contains('서울')]
seoul_2014_df.shape

(956287, 39)

In [51]:
seoul_2014_df = seoul_2014_df[colnames]
print(seoul_2014_df.shape)

(956287, 33)


In [52]:
seoul_2014_df.to_csv(basedir + 'floor_info_seoul_2014.csv', index=False)

In [53]:
%%time
floor_info_2017_df = pd.read_csv(basedir + 'mart_djy_04_201712.txt', encoding='euc-kr', sep='|', header=0, names=colnames)
print(floor_info_2017_df.shape)
floor_info_2017_df.head()

<string>:2: DtypeWarning: Columns (10,11,22) have mixed types. Specify dtype option on import or set low_memory=False.


(19523850, 33)
Wall time: 53.5 s


,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,29155-14451,광주광역시 남구 주월동 974-81번지,광주광역시 남구 군분로23번길 8-1,NaN,29155,10600.0,0.0,974,81,NaN,NaN,NaN,2.915543e+11,10601.0,0,8.0,1.0,NaN,20.0,지상,1,1층,11,벽돌구조,시멘트벽돌,01003,다가구주택,1가구 주택,101.89,0.0,주건축물,NaN,20100105
1,29155-14451,광주광역시 남구 주월동 974-81번지,광주광역시 남구 군분로23번길 8-1,NaN,29155,10600.0,0.0,974,81,NaN,NaN,NaN,2.915543e+11,10601.0,0,8.0,1.0,NaN,20.0,지상,2,2층,11,벽돌구조,시멘트벽돌,01003,다가구주택,2가구 주택,92.97,0.0,주건축물,NaN,20100105
2,29155-8546,광주광역시 남구 양촌동 358번지,광주광역시 남구 포충로 125-5,NaN,29155,11700.0,0.0,358,0,NaN,NaN,NaN,2.915532e+11,11701.0,0,125.0,5.0,NaN,20.0,지상,1,1층,11,벽돌구조,세멘벽돌,01001,단독주택,주택,59.72,0.0,주건축물,NaN,20100105
3,29155-40705,광주광역시 남구 주월동 997-7번지,광주광역시 남구 군분로77번안길 29-2,NaN,29155,10600.0,0.0,997,7,NaN,NaN,NaN,2.915543e+11,10602.0,0,29.0,2.0,NaN,20.0,지상,2,2층,11,벽돌구조,세벽,01001,단독주택,주택,52.24,0.0,주건축물,NaN,20100105
4,29155-40705,광주광역시 남구 주월동 997-7번지,광주광역시 남구 군분로77번안길 29-2,NaN,29155,10600.0,0.0,997,7,NaN,NaN,NaN,2.915543e+11,10602.0,0,29.0,2.0,NaN,20.0,지상,1,1층,11,벽돌구조,세벽,01001,단독주택,"주택,점포",78.11,0.0,주건축물,NaN,20100105


In [54]:
seoul_2017_df = floor_info_2017_df[floor_info_2017_df['대지_위치'].str.contains('서울')]
print(seoul_2017_df.shape)
seoul_2017_df.head()

ValueError: cannot index with vector containing NA / NaN values

In [55]:
nan_df = floor_info_2017_df[floor_info_2017_df['대지_위치'].isna()]
print(nan_df.shape)
nan_df.head()

(37, 33)


,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
118973,27710-15666,NaN,NaN,NaN,27710,32028.0,0.0,650,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,20.0,지상,1,1층,41,철골콘크리트구조,철근콘크리트조,01001,단독주택,단독주택(1),81.75,0.0,주건축물,NaN,20171205
118974,27710-15666,NaN,NaN,NaN,27710,32028.0,0.0,650,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,20.0,지상,2,2층,41,철골콘크리트구조,철근콘크리트조,01001,단독주택,단독주택(1),81.75,0.0,주건축물,NaN,20171205
118975,27710-15666,NaN,NaN,NaN,27710,32028.0,0.0,650,1,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,20.0,지상,3,3층,41,철골콘크리트구조,철근콘크리트조,01001,단독주택,단독주택(1),81.75,0.0,주건축물,NaN,20171205
857524,41461-26607,NaN,NaN,대성맨션,41461,38000.0,0.0,598,2,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,10.0,지하,1,지1층,21,철근콘크리트구조,철근콘크리트구조,02001,아파트,대피소,194.58,0.0,주건축물,NaN,20171205
857525,41461-26607,NaN,NaN,대성맨션,41461,38000.0,0.0,598,2,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,20.0,지상,1,1층,21,철근콘크리트구조,철근콘크리트구조,02002,연립주택,연립주택,180.00,0.0,주건축물,NaN,20171205


In [56]:
floor_info_2017_df.drop(nan_df.index, inplace=True)

In [57]:
seoul_2017_df = floor_info_2017_df[floor_info_2017_df['대지_위치'].str.contains('서울')]
print(seoul_2017_df.shape)
seoul_2017_df.head()

(2952425, 33)


,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
65,11545-6696,서울특별시 금천구 시흥동 860-48번지,서울특별시 금천구 독산로 151-4,NaN,11545,10300.0,0.0,860,48,NaN,NaN,NaN,1.154531e+11,10302.0,0,151.0,4.0,NaN,10.0,지하,1,지1,21,철근콘크리트구조,철근콘크리트조,01003,다가구주택,다가구주택(2가구),55.48,0.0,주건축물,NaN,20130219
66,11545-6696,서울특별시 금천구 시흥동 860-48번지,서울특별시 금천구 독산로 151-4,NaN,11545,10300.0,0.0,860,48,NaN,NaN,NaN,1.154531e+11,10302.0,0,151.0,4.0,NaN,20.0,지상,1,1층,11,벽돌구조,연와조,01003,다가구주택,다가구주택(2가구),55.48,0.0,주건축물,NaN,20130219
67,11545-6696,서울특별시 금천구 시흥동 860-48번지,서울특별시 금천구 독산로 151-4,NaN,11545,10300.0,0.0,860,48,NaN,NaN,NaN,1.154531e+11,10302.0,0,151.0,4.0,NaN,20.0,지상,2,2층,11,벽돌구조,연와조,01003,다가구주택,다가구주택(1가구),55.48,0.0,주건축물,NaN,20130219
68,11545-6696,서울특별시 금천구 시흥동 860-48번지,서울특별시 금천구 독산로 151-4,NaN,11545,10300.0,0.0,860,48,NaN,NaN,NaN,1.154531e+11,10302.0,0,151.0,4.0,NaN,30.0,옥탑,1,옥탑,11,벽돌구조,연와조,01003,다가구주택,물탱크실(연면적제외),6.76,0.0,주건축물,NaN,20130219
149,11350-4994,서울특별시 노원구 중계동 28-18번지,서울특별시 노원구 중계로 56-13,NaN,11350,10600.0,0.0,28,18,NaN,NaN,NaN,1.135031e+11,10603.0,0,56.0,13.0,NaN,20.0,지상,1,1층,12,블록구조,세멘부로크조,01001,단독주택,주택,20.00,0.0,주건축물,NaN,20130308


In [58]:
%%time
seoul_2017_df.to_csv(basedir + 'floor_info_seoul_2017.csv', index=False)

Wall time: 36.3 s


In [64]:
no_header_files_list[0].replace('.txt', '')[-6:-2]

'2015'

In [60]:
no_header_files_list

['MART_DJY_FLR_OULN_201512.txt',
 'mart_djy_04_201612.txt',
 'mart_djy_04_201812.txt',
 'mart_djy_04_201912.txt',
 'mart_djy_04_202011.txt']

In [65]:
for file in tqdm(no_header_files_list):
    year = file.replace('.txt', '')[-6:-2]
    try:
        df = pd.read_csv(basedir + file, encoding='euc-kr', sep='|', header=None, names=colnames)
    except:
        try:
            df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames)
        except:
            raise ValueError(file)
    df.dropna(subset=['대지_위치'], inplace=True)
    seoul_df = df[df['대지_위치'].str.contains('서울')]
    seoul_df.to_csv(basedir + 'floor_info_seoul_{}.csv'.format(year), index=False)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (2,9,10,11,22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 20%|████████████████▌                                                                  | 1/5 [01:58<07:54, 118.59s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 40%|█████████████████████████████████▏                                                 | 2/5 [03:43<05:31, 110.61s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,22) have mixed types. Specify